**silver facility momthly transactions**

In [0]:
from pyspark.sql import functions as F, Window

tx = spark.table("tp_finance.bronze.lti_facility_transactions")

# 1) Normalize to month and sign movement
tx_m = (
  tx.withColumn("month", F.date_trunc("month", F.col("transaction_date")).cast("date"))
    .withColumn(
      "movement",
      F.when(F.col("transaction_type") == "DRAW", F.col("amount"))
       .when(F.col("transaction_type") == "REPAY", -F.col("amount"))
       .otherwise(F.lit(0).cast("decimal(18,2)"))
    )
)

# 2) Monthly aggregates
monthly = (
  tx_m.groupBy("customer_id","contract_id","facility_id","month")
      .agg(
        F.sum(F.when(F.col("transaction_type")=="DRAW", F.col("amount")).otherwise(F.lit(0))).cast("decimal(18,2)").alias("drawn_this_month"),
        F.sum(F.when(F.col("transaction_type")=="REPAY", F.col("amount")).otherwise(F.lit(0))).cast("decimal(18,2)").alias("repaid_this_month"),
        F.sum("movement").cast("decimal(18,2)").alias("net_movement"),
        F.min("transaction_date").alias("min_tx_date"),
        F.max("transaction_date").alias("max_tx_date"),
        F.first("source_system").alias("source_system"),
      )
)

# 3) Generate a complete month series per facility (fill missing months)
facility_bounds = (
  monthly.groupBy("customer_id","contract_id","facility_id")
         .agg(
           F.min("month").alias("min_month"),
           F.max("month").alias("max_month"),
           F.first("source_system").alias("source_system")
         )
)

calendar = (
  facility_bounds
    .withColumn("month", F.explode(F.sequence(F.col("min_month"), F.col("max_month"), F.expr("interval 1 month"))))
    .select("customer_id","contract_id","facility_id","source_system","month")
)

# 4) Left join monthly movements onto full calendar; missing months become zeros
full = (
  calendar.join(monthly.select("customer_id","contract_id","facility_id","month","drawn_this_month","repaid_this_month","net_movement","min_tx_date","max_tx_date"),
                on=["customer_id","contract_id","facility_id","month"], how="left")
          .fillna({"drawn_this_month": 0, "repaid_this_month": 0, "net_movement": 0})
)

# 5) Compute running closing balance and opening balance (carry forward)
w = (Window.partitionBy("customer_id","contract_id","facility_id")
           .orderBy("month")
           .rowsBetween(Window.unboundedPreceding, Window.currentRow))

with_bal = (
  full
    .withColumn("closing_balance", F.sum("net_movement").over(w).cast("decimal(18,2)"))
    .withColumn("opening_balance", (F.col("closing_balance") - F.col("net_movement")).cast("decimal(18,2)"))
    .withColumn("load_ts", F.current_timestamp())
)

# 6) Upsert into Silver (rebuild per facility range)
with_bal.createOrReplaceTempView("stg_fac_m")

spark.sql("""
MERGE INTO tp_finance.silver.facility_monthly_balance t
USING stg_fac_m s
ON  t.customer_id = s.customer_id
AND t.contract_id = s.contract_id
AND t.facility_id = s.facility_id
AND t.month       = s.month
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
""")